In [5]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.datasets import CocoDetection
from torch.utils.data import DataLoader
from torchvision import transforms



## CNNモデルの作成

In [ ]:
#データセットの定義（ここではCOCOデータセットを例としています）
# チェックマークのデータセットを使用する場合は、この部分を適切に書き換える必要があります
dataset = CocoDetection(
    root = "path_to_your_dataset",
    annFile = "path_to_your_annotation_file",
    transform=transforms.ToTensor()
)

data_loader = DataLoader(
    dataset, 
    batch_size=2, 
    shuffle=True, 
    num_workers=4, 
    collate_fn=lambda x: tuple(zip(*x))
)

In [ ]:
#モデルの設定

# モデルの定義
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# 分類器の出力数をチェックマークのクラス数に合わせて変更
num_classes = 2  # 1 class (checkmark) + background
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


In [ ]:
#学習の設定
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

num_epochs = 10

In [ ]:
#学習の実行
for epoch in range(num_epochs):
    model.train()
    i = 0    
    for imgs, annotations in data_loader:
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        loss_dict = model(imgs, annotations)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        i += 1
        if i % 50 == 0:
            print(f"Iteration #{i} loss: {losses.item()}")

In [ ]:
# 訓練済みモデルの保存
torch.save(model.state_dict(), "path_to_save_your_model.pth")

## モデルを利用したチェックマークの検出